<a href="https://colab.research.google.com/github/NataliaRusinchuk/Numerical-methods-for-students/blob/main/Numerical%20Methods%201.2%20Solving%20the%20systems%20of%20equations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classes for systems of equations

In [ ]:
import numpy as np
import time
from scipy.misc import derivative


class SystemEq():
  def __init__(self, A, B):
    """
    A - np.ndarray
        matrix of coefficients
    B - matrix вільних членів
    """
    self.A = A
    self.B = B
    # Number of equations in the system
    self.length = self.B.size

  def function_iter(self, i):
    return lambda X: (- np.sum(self.A[i]*X) + self.B[i]) / self.A[i,i] + X[i]
  
  def simple_iter(self, maxiter = 1000, error = 1e-5):
    X0 = np.ones((self.length))
    X1 = np.zeros((self.length))
    answer = False
    for _ in range(maxiter):
      for i in range(self.length):
        X1[i] = self.function_iter(i)(X0)
        #print(X1-X0)
      if abs(max(X1-X0)) < error:
        answer = True
        break
      elif abs(min(X1-X0)) > 1e6:
        break
      X0 = np.array(X1)
    if answer:
      return (X1)
    else:
      print("Method could not solve the system")
      return (None)


  def execution_time(self, method):
    t_start = time.time()
    for _ in range(1000):
        method()
    t_end = time.time()
    return (t_end - t_start)



class LinearSystem(SystemEq):

  def gauss(self):
    return np.linalg.solve(self.A, self.B)



class NonLinearSystem(SystemEq):
  def __init__(self, functions):
    self.functions = functions
    self.length = len(functions)
    self.A = None
    self.B = None

  def function_iter(self, i, kof=100):
    return lambda X: X[i] - self.functions[i](X)/kof

  def newton(self, maxiter = 1000, error = 1e-5):
    X0 = np.ones((self.length))
    X1 = np.zeros((self.length))
    answer = False
    for _ in range(maxiter):
      J = np.zeros((self.length, self.length))
      F = np.zeros((self.length))
      for i in range(self.length):
        F[i] = self.functions[i](X0)
        for j in range(self.length):
          J[i,j] = derivative(
              # Функції є функціями від векторів
              # Для диференціювання потрібні функції від змінних
              # np.concatenate((X0[:j], [x], X0[j+1:])) - вектор, 
              # де всі відомі крім одного: спочатку [x, 1, 1],
              # потім [1, y, 1], потім [1, 1, z]
              lambda x: self.functions[i](np.concatenate((X0[:j], [x], X0[j+1:]))), 
              X0[j], 
              0.001
          )
      try:
        X1 = X0 - np.matmul(np.linalg.inv(J), F)
      except Exception as e:
        print(e)
        break
      if abs(max(X1-X0)) < error:
        answer = True
        break
      elif abs(min(X1-X0)) > 1e6:
        break
      X0 = np.array(X1)
    if answer:
      return (X1)
    else:
      print("Method could not solve the system")
      return (None)

  def simple_newton(self, maxiter = 1000, error = 1e-5):
    X0 = np.ones((self.length))
    X1 = np.zeros((self.length))
    answer = False
    J = np.zeros((self.length, self.length))
    for i in range(self.length):
      for j in range(self.length):
          J[i,j] = derivative(
              lambda x: self.functions[i](np.concatenate((X0[:j], [x], X0[j+1:]))), 
              X0[j], 
              0.001
          )
    for _ in range(maxiter):
      F = np.zeros((self.length))
      for i in range(self.length):
        F[i] = self.functions[i](X0)  
      try:
        X1 = X0 - np.matmul(np.linalg.inv(J), F)
      except Exception as e:
        print(e)
        break
      if abs(max(X1-X0)) < error:
        answer = True
        break
      elif abs(min(X1-X0)) > 1e6:
        break
      X0 = np.array(X1)
    if answer:
      return (X1)
    else:
      print("Method could not solve the system")
      return (None)

# Examples

In [ ]:
A = np.array([
    [10, 2, 0], 
    [5, -30, 1], 
    [1, 2, 30]
])
B = np.array([9, 7, 12])
eq_linear = LinearSystem(A, B)

In [ ]:
eq_linear.gauss()

array([ 0.91371539, -0.06857695,  0.37411462])

In [ ]:
eq_linear.simple_iter()

array([ 0.91371553, -0.06857521,  0.37411434])

In [ ]:
eq_linear.execution_time(eq_linear.gauss)

0.012694835662841797

In [ ]:
eq_linear.execution_time(eq_linear.simple_iter)

0.25916051864624023

# Linear random systems

In [ ]:
size = 10
A = np.random.rand(size, size)
B = np.random.rand(size)
system = LinearSystem(A, B)
system.execution_time(system.gauss)

0.01887369155883789

In [ ]:
# Метод не може бути застосований до цієї системи
system.simple_iter()

Method could not solve the system


In [ ]:
size = 20
A = np.random.rand(size, size)
B = np.random.rand(size)
system = LinearSystem(A, B)
system.execution_time(system.gauss)

0.04140162467956543

In [ ]:
size = 100
A = np.random.rand(size, size)
B = np.random.rand(size)
system = LinearSystem(A, B)
system.execution_time(system.gauss)

0.2253735065460205

# Linear diagonal systems

In [ ]:
size = 10
A = np.random.rand(size, size)
for i in range(size):
  A[i, i] *= 100
B = np.random.rand(size)
system = LinearSystem(A, B)
system.execution_time(system.gauss)

0.0153350830078125

In [ ]:
# Таку систему ми можемо розв'язати
system.simple_iter()

array([ 0.01349859,  0.00097851, -0.00075072,  0.0034292 ,  0.00167476,
        0.0162442 ,  0.03482978, -0.00104137,  0.0044496 ,  0.0261682 ])

In [ ]:
# Перевіримо час виконання
system.execution_time(system.simple_iter)

0.838782548904419

In [ ]:
size = 20
A = np.random.rand(size, size)
for i in range(size):
  A[i, i] *= 100
B = np.random.rand(size)
system = LinearSystem(A, B)
system.execution_time(system.simple_iter)

2.9779515266418457

In [ ]:
size = 100
A = np.random.rand(size, size)
for i in range(size):
  A[i, i] *= 1000
B = np.random.rand(size)
system = LinearSystem(A, B)
system.execution_time(system.simple_iter)

15.343128204345703

# Nonlinear system

In [ ]:
# Задаємо функції як залежні від вектору
# (x, y, z) = X
# x = X[0]
# y = X[1]
# z = X[2]

def eq_1(X):
  return X[0]**2 - 400*X[0]*X[1] - 1200

def eq_2(X):
  return X[0]*X[1] - 40*X[0]*X[2] - 80

def eq_3(X):
  return X[0] + 20*X[1] + 50*X[0]*np.sqrt(abs(X[2])) - 45

eqs = (eq_1, eq_2, eq_3)

nonlin_eq = NonLinearSystem(eqs)

In [ ]:
nonlin_eq.simple_iter()

Method could not solve the system


In [ ]:
nonlin_eq.newton()

array([ 1.41898879, -2.11063413, -1.46222026])

In [ ]:
nonlin_eq.execution_time(nonlin_eq.newton)

5.258514165878296

In [ ]:
nonlin_eq.simple_newton()

Method could not solve the system
